This example simulates the build up of activate products within a material under neutron irradiation. The subsequent decay of unstable isotopes is also simulated.

This first cell imports the packages needed, note the extra import openmc.deplete import

In [ ]:
import openmc
import openmc.deplete
import os

# this command makes use of a terminal command from the openmc_data package
# more info on openmc_data package can be found here https://github.com/openmc-data-storage/openmc_data
# this command downloads a chain file 
# the chain file has information on half-lives, branching rations and gamma spec of isotopes

os.system('download_nndc_chain')


This section creates the geometry and the cells.
Note that it it necessary to set the volume of the material or cell.
This is so that the depletion code can find the number of atoms within the cell given the material composition, material density and volume.

In [ ]:

import math

# MATERIALS

mats = openmc.Materials()

# makes a simple material from Silver
my_material = openmc.Material() 
my_material.add_element('Ag', 1, percent_type='ao')
my_material.set_density('g/cm3', 10.49)


sphere_radius = 100
volume_of_sphere = (4/3) * math.pi * math.pow(sphere_radius, 3)
my_material.volume = volume_of_sphere  # a volume is needed so openmc can find the number of atoms in the cell/material
my_material.depletable = True  # depletable = True is needed to tell openmc to update the material with each time step

materials = openmc.Materials([my_material])
materials.export_to_xml()


# GEOMETRY

# surfaces
sph1 = openmc.Sphere(r=sphere_radius, boundary_type='vacuum')

# cells, makes a simple sphere cell
shield_cell = openmc.Cell(region=-sph1)
shield_cell.fill = my_material

# sets the geometry to the universe that contains just the one cell
universe = openmc.Universe(cells=[shield_cell])
geometry = openmc.Geometry(universe)



This section defines the neutron source term to use and the settings

In [ ]:
# creates a 14MeV neutron point source
source = openmc.Source()
source.space = openmc.stats.Point((0, 0, 0))
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete([14e6], [1])
source.particles = 'neutron'

# SETTINGS

# Instantiate a Settings object
settings = openmc.Settings()
settings.batches = 2
settings.inactive = 0
settings.particles = 10000
settings.source = source
settings.run_mode = 'fixed source'

model = openmc.model.Model(geometry, materials, settings)

This is the depletion specific part of the model setup.

This section specifies the chain file, this tells openmc the decay paths between isotopes including probabilities of different routes and half lives

In [ ]:
# This chain_endfb71 file was downloaded using the download_nndc_chain script that is included in the openmc_data package https://github.com/openmc-data-storage/openmc_data\n",
# this file tells openmc the decay paths between isotopes including probabilities of different routes and half lives
openmc.config['chain_file'] = 'chain-nndc-b7.1.xml'

operator = openmc.deplete.Operator(
    model=model,
    normalization_mode="source-rate",  # set for fixed source simulation, otherwise defaults to fission simulation
    dilute_initial=0,  # set to zero to avoid adding small amounts of isotopes, defaults to adding small amounts of fissionable isotopes
    reduce_chain=True,  # reduced to only the isotopes present in depletable materials and their possible progeny
    reduce_chain_level=5,
)



This next stage sets the time steps and corresponding source rates for the irradiation schedule.

An output file will be produced with showing the material composition at every time step.

We are irradiating the Silver for multiple half lives to show build up and saturation

Saturation happens when decay is = to creation of the particular isotope

Ag110 half life is 24 seconds so it will start to become saturated after 120 seconds

Ag108 half life is 145 seconds so it will not be saturated

In [ ]:
# We define timesteps together with the source rate to make it clearer
timesteps_and_source_rates = [
    (24, 1e20),
    (24, 1e20),
    (24, 1e20),
    (24, 1e20),
    (24, 1e20),  # should saturate Ag110 here as it has been irradiated for over 5 halflives
    (24, 1e20),
    (24, 1e20),
    (24, 1e20),
    (24, 1e20),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
]

# Uses list Python comprehension to get the timesteps and source_rates separately
timesteps = [item[0] for item in timesteps_and_source_rates]
source_rates = [item[1] for item in timesteps_and_source_rates]


# PredictorIntegrator has been selected as the depletion operator for this example as it is a fast first order Integrator
# OpenMC offers several time-integration algorithms https://docs.openmc.org/en/stable/pythonapi/deplete.html#primary-api\n",
# CF4Integrator should normally be selected as it appears to be the most accurate https://dspace.mit.edu/handle/1721.1/113721\n",
integrator = openmc.deplete.PredictorIntegrator(
    operator=operator,
    timesteps=timesteps,
    source_rates=source_rates
)


This next section starts the depletion simulation and produces the output files

In [ ]:

integrator.integrate()

# bash command to show the output files produce
!ls -lh *.h5

This section extracts the results of the depletion simulation from the h5 file and gets the amount of Ag110 in the material at each of the time steps

In [ ]:
results = openmc.deplete.ResultsList.from_hdf5("depletion_results.h5")

times, number_of_Ag110_atoms = results.get_atoms(my_material, 'Ag110')

for time, num in zip(times, number_of_Ag110_atoms):
    print(f" Time {time}s. Number of Ag110 atoms {num}")

In addition to Ag110 other atoms get created. This section plots the number of nuclides in the material excluding the original nuclides in the unirradiated material

In [ ]:
import openmc_depletion_plotter
# this package provides convenient plotting methods for depletion simulations like this one
# more details here https://github.com/fusion-energy/openmc_depletion_plotter

results.plot_atoms_vs_time(excluded_material=my_material)

Not all nuclide are unstable and the unstable ones have a different half life. This next plot shows the specific activity (activity per unit mass) as a function of time.

This is useful for identifying a suitable waste repository for activated waste.

In [ ]:
results.plot_activity_vs_time()